In [ ]:
from google.colab import drive
drive.mount('/content/drive')
%cd /content/drive/MyDrive/Colab-LLMVS/


Mounted at /content/drive
/content/drive/MyDrive/Colab-LLMVS


In [ ]:
# In Google Colab - install only non-PyTorch dependencies
!pip install numpy scipy pandas einops torchmetrics
!pip install matplotlib tqdm tensorboard h5py
!pip install pytorch-lightning>=2.0.0


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 983.2/983.2 kB 53.9 MB/s eta 0:00:00


In [ ]:
# run this BEFORE train.py
import os

# Create checkpoint directories
os.makedirs('Summaries/tvsum_head2_layer3/tvsum/tvsum_split0/best_rho_model', exist_ok=True)
os.makedirs('Summaries/tvsum_head2_layer3/tvsum/tvsum_split0/best_tau_model', exist_ok=True)

print("✓ Checkpoint directories created")

✓ Checkpoint directories created


In [ ]:
# Training one split only
!python train.py --tag tvsum_split0 --model tvsum_head2_layer3 --lr 0.00007 --epochs 200 --dataset tvsum --reduced_dim 2048 --num_heads 2 --num_layers 3 --split_idx 0 --pt_path 'llama_emb/tvsum_sum/'



Streaming output truncated to the last 5000 lines.
Epoch 171/199 ━━━━━━━━━━━━╺━━━ 31/40 0:00:19 •        1.54it/s v_num: 25.000    
                                     0:00:06                   val_kTau: 0.177  
Epoch 171/199 ━━━━━━━━━━━━╸━━━ 32/40 0:00:19 •        1.58it/s v_num: 25.000    
                                     0:00:06                   val_kTau: 0.177  
Epoch 171/199 ━━━━━━━━━━━━━╺━━ 33/40 0:00:19 •        1.63it/s v_num: 25.000    
                                     0:00:05                   val_kTau: 0.177  
Epoch 171/199 ━━━━━━━━━━━━━╸━━ 34/40 0:00:19 •        1.68it/s v_num: 25.000    
                                     0:00:04                   val_kTau: 0.177  
Epoch 171/199 ━━━━━━━━━━━━━━╺━ 35/40 0:00:19 •        1.73it/s v_num: 25.000    
                                     0:00:03                   val_kTau: 0.177  
Epoch 171/199 ━━━━━━━━━━━━━━╺━ 36/40 0:00:19 •        1.78it/s v_num: 25.000    
                                     0:00:03              

In [ ]:
%%shell
cp -r /content/best_rho_model /content/drive/MyDrive/Colab-LLMVS/Summaries/
cp -r /content/best_tau_model /content/drive/MyDrive/Colab-LLMVS/Summaries/

In [ ]:
!python test_splits.py --dataset tvsum --weights rho

Seed set to 1112
Traceback (most recent call last):
  File "/content/drive/MyDrive/Colab-LLMVS/test_splits.py", line 40, in <module>
    --pt_path {} --num_heads {} --num_layers {} >> {}".format(args.model, args.dataset, tags[i], args.reduced_dim, i, weights[i], args.result_dir, args.pt_path, args.num_heads, args.num_layers, file_name))
                                                                                        ~~~~^^^
IndexError: list index out of range


In [ ]:
%cd /content/drive/MyDrive/Colab-LLMVS
!python test.py \
    --dataset tvsum \
    --split_idx 0 \
    --weights Summaries/best_rho_model/epoch=143-val_sRho=0.232.ckpt \
    --pt_path llama_emb/tvsum_sum/

/content/drive/MyDrive/Colab-LLMVS
Seed set to 1112
Seed set to 1112
Device being used: cuda:0
/usr/local/lib/python3.12/dist-packages/torch/utils/data/dataloader.py:627: UserWarning: This DataLoader will create 32 worker processes in total. Our suggested max number of worker in current system is 12, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(
/usr/local/lib/python3.12/dist-packages/lightning_fabric/connector.py:571: `precision=16` is supported for historical reasons but its usage is discouraged. Please set your precision to 16-mixed instead!
Using 16bit Automatic Mixed Precision (AMP)
💡 Tip: For seamless cloud uploads and versioning, try installing [litmodels](https://pypi.org/project/litmodels/) to enable LitModelCheckpoint, which syncs automatically with the Lightning model regist

In [ ]:
%cd /content/drive/MyDrive/Colab-LLMVS

!python save_frame_indices.py \
    --weights Summaries/tvsum_head2_layer3/tvsum/tvsum_split0/best_rho_model/epoch=53-val_sRho=0.230.ckpt \
    --pt_path llama_emb/tvsum_sum/ \
    --dataset tvsum \
    --split_idx 0 \
    --model tvsum_head2_layer3 \
    --tag tvsum_split0 \
    --output_json tvsum_frame_indices_with_scores.json

/content/drive/MyDrive/Colab-LLMVS
Seed set to 1112
Device being used: cuda:0
Loading dataset: tvsum...
Loading model from Summaries/tvsum_head2_layer3/tvsum/tvsum_split0/best_rho_model/epoch=53-val_sRho=0.230.ckpt...
Using device: cuda
Running inference...
100% 10/10 [00:01<00:00,  9.30it/s]
Successfully saved frame indices to tvsum_frame_indices_with_scores.json


In [ ]:
#The paths for videos has to be updated while running below
!python create_summary_video.py \
    --json_path tvsum_frame_indices_with_scores.json \
    --h5_path TVSum/eccv16_dataset_tvsum_google_pool5.h5 \
    --video_dir /Users/konda/Downloads/Fall_2025/NLP/Project2/Colab-LLMVS/datasets/tvsum50_ver_1_1/ydata-tvsum50-v1_1/video \
    --video_key video_45 \
    --output video_45_summary.mp4